# Transformer 

- [Attention Is All You Need 2017](https://arxiv.org/pdf/1706.03762) 논문에서 제시한 모델로 RNN모듈을 사용하지 않고 **Attention 모듈로만 구성한 Encoder-Decoder 구조의 seq2seq 모델**이다.
    - RNN → RNN + attention → Transformer
- Transformer 이후 자연어 관련 pretrained 모델들은 대부분 Transformer의 Encoder나 Decoder를 변형하거나 단순히 층을 깊게 쌓는 형태로 구현하여 성능을 올렸다.
    - Google의 BERT(Bidirectional Encoder Representations from Transformers)는 Transformer의 Encoder를 이용해 구현했다.
    - OpenAI의 GPT(Generative Pre-trained Transformer)는 Transformer의 Decoder를 이용해 구현했다.

## Seq2Seq의 문제와 Transformer
- Seq2Seq 의 문제
    - 장기기억 소실의 문제.(long term dependency problem)
        - seq가 길어지면 초기 timestep의 정보가 점점 소실된다.
    - 병렬처리를 못해 연산 효율이 떨어진다.
        - time step **순서대로** 처리해야 하기 때문에 병렬처리가 안된다.
- **장기기억 소실문제는** seq2seq에 **Attention mechanism**을 적용해 해결할 수 있다.
  
![img](figures/transformer_advantage.gif)
- Transformer는  encoder와 decoder에서 RNN을 제거하고 **Attention 모듈로 변경하여 병렬 처리가 가능**하도록 했다.

## Transformer Architecture
- Encoder - Decoder  구조
  - Encoder는 **Self-Attention + Position-wise FFN**으로
  - Decoder는 **Masked Self-Attention → Encoder-Decoder Cross-Attention → FFN**의 구조로 한개 Layer(층)을 이룬다.
- 각 Layer는 Residual Connection(Skip Connection)과 Layer Normalization 을 이용해 깊은 Layer를 쌓을 수 있도록 한다.

- **Positional Encoding**
    - RNN을 사용하지 않기 때문에 입력 sequence의 토큰들에 순서정보가 없다. Positional Encoding은 를 Embedding Vector에 추가할 순서 정보를 생성한다.
    - 문장을 구성하는 각 토큰들의 embedding vector에 그 토큰 순서 정보를 가지는 positional encoding 값을 더해준다.
    - Positional Encoding 값은 sine/cosin 주기함수를 이용해 계산한다.
    - 대표적인 공식: $PE_{(pos,2i)}=\sin\left(\dfrac{pos}{10000^{2i/d_{model}}}\right)$, $PE_{(pos,2i+1)}=\cos\left(\dfrac{pos}{10000^{2i/d_{model}}}\right)$
      - $pos$: 입력 시퀀스에서 토큰 index
      - $i$: 0부터 $d_{model}/2-1$까지 번호. 이 번호로 차원 쌍(2i, 2i+1)을 만들고, 쌍의 앞은 sin, 뒤는 cos를 써서 같은 위치에 대해 두 값을 만든다. 두 함수를 함께 써야 위치가 겹치지 않는다. 
        - 예) (0, 1) 의 $i$: 0, (2, 3) 의 $i$는 1, ..
      - $d_{model}$: 임베딩 차원수

In [ ]:
import math

d_model = 1000  # Embedding vector의 차원수 (pe도 30차원)
pos_indices = range(5) # 토큰들의 index (0 ~ n)
pe_all = [] # 모든 토큰들(5개)를 위한 positional encoding을 저장할 리스트
for pos in pos_indices:
    pe = [] # 개별 토큰의 positional encoding을 저장할 리스트.
    
    for i in range(d_model // 2): #0-(0:sin, 1:cos), 1-(2, 3), 2-(4, 5)
        div = 10000 ** (2*i / d_model)# 분모
        pe.append(math.sin(pos / div)) # 짝수(쌍에서 앞) index
        pe.append(math.cos(pos / div)) # 홀수(쌍에서 뒤) index
    
    pe_all.append(pe)

In [7]:
pe_all

[[0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,

![img](figures/transformer_architecture1.png)

\[**Encoder와 Decoder의 구조**\]

![img](figures/transformer_architecture2.png)

\[**transformer의 다층 구조**\]


## Self Attention
- Attention을 자기 자신한테 적용한다는 의미
    - seq2seq 의 attention은 decoder의 sequence가 encoder의 hidden state에 attention을 취한다. (디코더가 인코더의 어디에 집중할 지 찾는다.)
    - self attention은 attention을 자기 자신에게 취한다. (같은 문장 내에서 단어들이 서로 어떤 관계를 가지는지 찾는다.)
- Self-attention은 하나의 단어가 자신이 속한 문장의 다른 단어들과 어떤 연관성을 가지는지 찾는다.

  ![img](figures/transformer_self-attention.png)

- 예를 들어 "The animal didn't cross the street because it was too tired." 라는 문서에 있는  `it`을 encoding 한려고 한다. 
- `it`을 문맥에 맞게 embedding하기 위해서는 이 문서안에서 어떤 의미로 쓰였는지 알아야 한다. 
- 그것을 알기 위해서는 `it` 이 가리키는(관련있는) 것이 어떤 것인지 알아야 한다. 그것을 다른 문서에서 찾는 것이 아니라 `it` 있는(대상 단어가 있는) 문서에서 다른 단어들과의 연관성에서 찾는다.
- **의미를 파악하려는 문서와 그 의미를 찾는 문서가 동일하기 때문에 self-attention**이라고 한다.

### Self Attetion 과정
- 같은 문장내에서 단어들 간의 관계를 고려하는 것.
  
   ![self-attention](figures/transformer_self_attention.png)

- **수식**
\begin{align}
&\text{Attention(Q,K,V)} = \text{softmax}(\cfrac{QK^T}{\sqrt{d_k})})V \\
&\small \text{Q: Query, K: Key, V: Value,}\;d_{k}: \text{embedding 차원수}
\end{align}

## Multi-Head Attention에서 Q, K, V의 역할



#### **1. Query, Key, Value 생성**
- **Query (Q)**: 현재 위치에서 "무엇을 찾고 있는지"를 나타내는 벡터로, 다른 위치들과의 관련성을 질의하는 역할.
- **Key (K)**: 각 위치가"어떤 정보를 가지고 있는지"를 나타내는 벡터로, **Query와 매칭되어 attention score를 계산하는 기준**이 된다.
- **Value (V)**: 실제로 "**전달할 정보의 내용**"을 담고 있는 벡터로, attention weight와 가중합되어 최종 출력을 생성한다.  이 최종 출력값이 context vector(문맥 정보)가 된다.
- Self attention은 Query, Key, Value 모두 입력 Sequence(X)로 부터 생성한다.
    - 입력 sequence에 각각 query, key, value를 만드는 Weight들(Linear)를 내적하여 만든다.
      - $Query=X\cdot W_q$ 
      - $Key=X\cdot W_k$ 
      - $Value=X\cdot W_v$

  
  ![img](figures/transformer_query_key_value.png)

#### **2. Attention Score**
- Query 와 Key를 내적(dot product)하여 유사도를 계산한다. embedding vector의 차원의 제곱근으로 나눠서 정규화한다.
- Scaled Dot Product Attention
  
   $$
   \text{Attention Score} = \cfrac{Q\cdot K^T}{\sqrt{d_k}}
   $$
  ![img](figures/transformer_query_key_matmulpng.png)

#### 3. **Attention Weight**
   - 위에서 계산된 Attention score에 softmax를 적용해 0 ~ 1 사이 비율로 바꾼다.
     
   $$
    \text{Attention Weight} = softmax(\text{Attention Score})
   $$

#### **4. Attention value**
   - 최종 attention 연산의 결과로 **Attention weight를 Value에 내적**해서 Attention Value를 만든다.
   - Attention에서 Attention value값이 **입력 sequence의 context vector**가 된다.
   $$
   \text{Attention Value} = \text{Attention Weight}\cdot\text{Value}
   $$ 

   ![img](figures/transformer_attention_value.png)

## Multi-Head attention
- "Multi-Head Attention은 입력 데이터에 서로 다른 가중치 행렬을 적용하여 **여러 개의 Q, K, V 세트**를 생성하고, 각 세트마다 병렬적으로 attention을 수행한다."
- 이렇게 나눈 것을 head라고 하고 여러개를 만들어 사용하므로 multi-head attention이라고 한다.
- **Multi-Head 생성 과정**
    - 입력 Embedding Vector가 512차원이고 Head를 8개 사용한다고 할 경우
    - 각 Head는 입력 전체(512차원)에 대해 서로 다른 학습 가능한 가중치 행렬을 적용하여 64차원(512/8)의 독립적인 Query, Key, Value를 생성한다. (Single Head는 512차원 전체를 사용하여 512차원의 Query, Key, Value 생성)
        - 효율적인 병렬 처리를 위해 일반적으로 `d_model % head수 = 0`이 되도록 설정한다. (필수 조건은 아니지만 구현상 편의를 위함)
        - ex) head 개수: 8, embedding 차원: 512, head 차원: 64
            - 512 x 64 (embedding_dim x head_dim)의 가중치 행렬(Linear)를 8 개 생성하고 입력 데이터와 행렬곱을 통해 64차원의 Query 8개를 생성한다. Key와 Value도 각각의 같은 방식으로 8개씩 head를 생성한다.
    - 각 head는 독립적으로 Attention 연산을 수행한다. (**병렬 처리**)
    - 모든 head의 출력을 concat한 후(8×64 = 512차원), 최종적으로 출력 차원을 원하는 차원으로 매핑하기 위해 추가적인 선형 변환을 수행한다.

- **장점**
    - 동일한 입력 시퀀스에 대해 여러 관점(문법적 관계, 의미적 관계 등)에서의 정보를 동시에 학습할 수 있다.
    - 각 head가 서로 다른 특성을 학습하여 더 풍부한 표현이 가능하다.
    - 여러 head의 연산이 병렬적으로 처리될 수 있어 GPU와 같은 하드웨어에서 연산 속도를 극대화할 수 있다.


  <img src="figures/transformer_multi-head-attention.png" width="1300">

## Masked self Attention
- 입력 sequence 가 순서대로 입력되어 처리 되는 경우 i번째 입력 단어의 경우 그 이후의 단어는 모르는 상태이다. 
- Attention은 입력된 모든 토큰을 한번에 처리해서 attention score를 구한다. 이것은 주어진 토큰(i번째)이 미래시점의 입력토큰(i+1 이후 번째)과의 유사도를 계산한 것이 된다. 
- 이 문제를 해결하기 위해 i번째 토큰에 대한 attention score는 i번째 까지의 토큰들과만 계산하도록 한다.
- Attention Score를 Softmax 계산하기 전에 적용한다.
- Attention 계산시 선택적으로 mask를 사용하면 Masked self attention이 된다. (Masked Multi-Head Attention)
- **마스크 종류**
    - padding mask: PAD 토큰이 가중치에 반영되지 않도록 **encoder/decoder** 모두에서 사용.
    - causal mask: **decoder**에서 미래 토큰 정보를 차단해 auto-regressive 생성을 가능하게 함.
  
  ![img](figures/transformer_masked_self_attention.png)

## Point-wise Feed-Forward Network (FFN)

- **Transformer의 Encoder와 Decoder 내부에 동일한 구조로 반복 삽입되는 비선형 변환 블록**
- **각 토큰(position)마다 동일한 FFN이 독립적으로 적용되어**, 토큰 간 상호작용 없이 **개별 토큰의 표현을 비선형적으로 변환**하는 역할을 수행한다.

### 구조 (Structure)

```mermaid
flowchart LR
    A[입력 벡터<br/>차원: 512] --> B[Linear<br/>512 → 2048]
    B --> C[ReLU]
    C --> D[Linear<br/>2048 → 512]
    D --> E[출력 벡터<br/>차원: 512]
```

- **2개 Layerd의 Fead Forward Network(FFN) 구조**
  - 차원 확장 → 비선형 변환 → 차원 복원(축소) 구조를 가진다.
  - 모든 토큰들에 **동일한 가중치를 공유하며 독립적으로 적용 (Point-wise)** 한다.

### 목적 및 역할 (Purpose & Role)

- **Self-Attention을 통해 토큰 간 관계 기반의 문맥 정보가 결합된 벡터를 입력으로 받아,  각 토큰 자체의 의미를 비선형적으로 변환하고 정제하는 역할**을 담당한다.
  - **Self-Attention**
    - 토큰(단어) 간의 **관계 학습**
    - 문맥(Context) 정보 결합
  - **FFN**
    - 각 토큰 벡터에 대해 **비선형 변환 수행**
    - 개별 토큰의 **표현력(Representation Power) 증가**

### FFN이 필요한 이유

- **Attention Block은 기본적으로 선형 연산만 포함**. 선형 변환만으로는 **복잡한 의미 패턴과 고차원 표현 학습에 한계**
- FFN의 **ReLU(GELU) 기반 비선형 함수**를 통해
  - 모델의 **표현력(Expressiveness) 증가**
  - 복잡한 의미 구조 학습 가능


# Transfomer 모델의 구조적 분류와 대표모델

- Transfomer 모델을 기반으로 구현된 다양한 모델들이 있다. 
- 트랜스포머 모델은 구조에 따라 크게 세 가지로 분류할 수 있었다.

## 인코더 전용 모델 (Encoder-Only)
- Transformer Encoder 구조를 이용해 구현된 모델들. 
- **대표 모델들**:
    - BERT: 마스크드 언어 모델링과 다음 문장 예측을 목표로 하는 모델.
    - RoBERTa: BERT를 개선하여 더 많은 데이터와 더 큰 배치로 성능을 향상시킨 모델.
    - ALBERT: 파라미터 수를 줄이고 레이어 간 파라미터를 공유하도록 개선한 모델.

## 디코더 전용 모델 (Decoder-Only)
- Transformer Decoder 구조를 이용해 구현된 모델들. 
- **대표 모델들**:
    - GPT 시리즈: 텍스트 생성에 특화된 모델로, GPT-1부터 GPT-3까지 발전했다
    - CTRL: 제어 토큰을 추가하여 생성 문장의 스타일을 제어할 수 있는 모델
    - OPT: GPT와 유사한 구조를 가진 오픈 소스 모델

## 인코더-디코더 모델 (Encoder-Decoder)
- Transformer의 Encoder-Decoder 구조를 이용해 구현된 모델들.
- **대표 모델들**:
    - T5: 모든 NLP 태스크를 텍스트-투-텍스트 형식으로 변환하여 처리하는 모델
    - BART: BERT의 인코더와 GPT의 디코더를 결합한 모델
    - M2M-100: 100개 언어 간 번역이 가능한 다국어 번역 모델
    - BigBird: 긴 시퀀스를 처리할 수 있도록 개선된 모델

![transformer_models.png](figures/transformer_models.png)

<대표적인 transformers 기반 모델들>